In [1]:
def extract_features(packet):
    features = [None] * 17
    if IP in packet:
        ip = packet[IP]
        features[0] = ip.version
        features[1] = ip.ihl
        features[2] = ip.tos
        features[3] = ip.len
        features[4] = ip.flags
        features[5] = ip.frag
        features[6] = ip.ttl
        features[7] = ip.proto
    if TCP in packet:
        tcp = packet[TCP]
        features[8] = tcp.sport
        features[9] = tcp.dport
        features[10] = tcp.seq
        features[11] = tcp.ack
        features[12] = tcp.flags
        features[13] = tcp.window
    if UDP in packet:
        udp = packet[UDP]
        features[14] = udp.sport
        features[15] = udp.dport
        features[16] = udp.len
    return features

In [2]:
from scapy.all import *
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from scapy.layers.inet import IP, TCP, UDP

# Read and parse malware dataset
malware_dir = 'C:/Users/14802/Desktop/Mal'
malware_data = []
for file in os.listdir(malware_dir):
    if file.endswith('.pcap'):
        file_path = os.path.join(malware_dir, file)
        try:
            packets = rdpcap(file_path)
            malware_data.extend([(packet, 1) for packet in packets])
        except Exception as e:
            print(f"Error reading file {file}: {str(e)}")

# Read and parse benign network traffic dataset
traffic_path = 'C:/Users/14802/Desktop/Traffic/traffic.pcapng'
try:
    packets = rdpcap(traffic_path)
    print(f"File {traffic_path} has {len(packets)} packets")
    malware_data.extend([(packet, 0) for packet in packets])
except Exception as e:
    print(f"Error reading file {traffic_path}: {str(e)}")

# Split into training and testing set
train_data, test_data = train_test_split(malware_data, test_size=0.2, random_state=42)

# Extract features from the packets
def extract_features(packet):
    features = []
    if IP in packet:
        ip = packet[IP]
        features.extend([ip.version, ip.ihl, ip.tos, ip.len, ip.flags, ip.frag, ip.ttl, ip.proto])
    if TCP in packet:
        tcp = packet[TCP]
        features.extend([tcp.sport, tcp.dport, tcp.seq, tcp.ack, tcp.flags, tcp.window])
    if UDP in packet:
        udp = packet[UDP]
        features.extend([udp.sport, udp.dport, udp.len])
    return features

# Extract features and labels from the training and testing data
X_train = [extract_features(packet) for packet, _ in train_data]
y_train = [label for _, label in train_data]
X_test = [extract_features(packet) for packet, _ in test_data]
y_test = [label for _, label in test_data]

# Train a random forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Test the accuracy of the model on the test set
y_pred = rf.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")

File C:/Users/14802/Desktop/Traffic/traffic.pcapng has 38642 packets


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (164707,) + inhomogeneous part.